In [ ]:
!pip install scikit-learn nltk python-Levenshtein spacy&> /dev/null

In [ ]:
import nltk
import spacy

nltk.download('punkt')
!python -m spacy download en_core_web_sm


Cosine Similarity

Jaccard Similarity

Levenshtein Distance

Difflib Sequence Matching

## Using Copyscape

In [ ]:

# Copyscape API credentials
USERNAME = 'your_username'
API_KEY = 'your_api_key'


In [ ]:
import requests

# Copyscape API URL
COPESCAPE_API_URL = 'https://www.copyscape.com/api/'

def copyscape_url_search(url, full_comparisons=0, ignore_sites=None, spend_limit=None):
    """
    Perform a URL search using the Copyscape API.
    """
    params = {
        'u': USERNAME,
        'k': API_KEY,
        'o': 'csearch',
        'q': url,
        'c': full_comparisons,
        'f': 'json'
    }

    if ignore_sites:
        params['i'] = ','.join(ignore_sites)

    if spend_limit:
        params['l'] = spend_limit

    response = requests.get(COPESCAPE_API_URL, params=params)
    return response.json()

def copyscape_text_search(text, full_comparisons=0, ignore_sites=None, spend_limit=None):
    """
    Perform a text search using the Copyscape API.
    """
    params = {
        'u': USERNAME,
        'k': API_KEY,
        'o': 'csearch',
        'e': 'UTF-8',
        'c': full_comparisons,
        'f': 'json'
    }

    if ignore_sites:
        params['i'] = ','.join(ignore_sites)

    if spend_limit:
        params['l'] = spend_limit

    response = requests.post(COPESCAPE_API_URL, params=params, data=text.encode('utf-8'))
    return response.json()

def add_to_private_index(url=None, text=None, article_id=None, article_title=None):
    """
    Add content to your private index using the Copyscape API.
    """
    params = {
        'u': USERNAME,
        'k': API_KEY,
        'o': 'pindexadd',
        'f': 'json'
    }

    if url:
        params['q'] = url
    elif text:
        params['t'] = text
        params['e'] = 'UTF-8'

    if article_id:
        params['i'] = article_id

    if article_title:
        params['a'] = article_title

    if url:
        response = requests.get(COPESCAPE_API_URL, params=params)
    else:
        response = requests.post(COPESCAPE_API_URL, params=params, data=text.encode('utf-8'))

    return response.json()

# Example usage
if __name__ == "__main__":
    # URL search example
    url_search_result = copyscape_url_search('http://example.com')
    print(url_search_result)

    # Text search example
    text_search_result = copyscape_text_search('This is a sample text to check for plagiarism.')
    print(text_search_result)

    # Add to private index example
    add_result = add_to_private_index(text='This is a sample text to add to my private index.')
    print(add_result)


In [2]:
# Import required libraries
from difflib import SequenceMatcher
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from typing import List, Tuple

# Function to calculate Levenshtein Distance
def levenshtein_distance(s1: str, s2: str) -> int:
    if len(s1) < len(s2):
        return levenshtein_distance(s2, s1)

    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row

    return previous_row[-1]

# Function to calculate Sequence Matching
def sequence_match_score(s1: str, s2: str) -> float:
    return SequenceMatcher(None, s1, s2).ratio()

# Function to calculate Jaccard Similarity
def jaccard_similarity(s1: str, s2: str) -> float:
    set1 = set(s1.split())
    set2 = set(s2.split())
    return len(set1 & set2) / len(set1 | set2)

# Function to calculate Cosine Similarity
def calculate_cosine_similarity(documents: List[str]) -> float:
    count_vectorizer = CountVectorizer().fit_transform(documents)
    vectors = count_vectorizer.toarray()
    return cosine_similarity(vectors)[0][1]

# Function to compare a document against multiple potential sources
def compare_documents(suspected_text: str, source_texts: List[str]) -> List[Tuple[str, float, float, float, float]]:
    results = []
    for source_text in source_texts:
        leven_distance = levenshtein_distance(suspected_text, source_text)
        seq_match = sequence_match_score(suspected_text, source_text)
        jaccard_sim = jaccard_similarity(suspected_text, source_text)
        cosine_sim = calculate_cosine_similarity([suspected_text, source_text])
        results.append((source_text, leven_distance, seq_match, jaccard_sim, cosine_sim))
    return results


# Example usage
suspected_text = "This is a sample text that might be plagiarized."
source_texts = [
    "This is a sample text that could be plagiarized.",
    "An original text that has nothing to do with the suspected text.",
    "This is another text that might be slightly similar to the suspected text.",
]

# Compare the suspected document against the source documents
comparison_results = compare_documents(suspected_text, source_texts)

comparison_results

[('This is a sample text that could be plagiarized.',
  5,
  0.8958333333333334,
  0.8,
  0.8749999999999999),
 ('An original text that has nothing to do with the suspected text.',
  47,
  0.44642857142857145,
  0.10526315789473684,
  0.2834733547569204),
 ('This is another text that might be slightly similar to the suspected text.',
  38,
  0.5901639344262295,
  0.375,
  0.6390096504226936)]

In [3]:
# Integration with the existing comparison code
def plagiarism_check_with_copyscape(suspected_text: str, source_texts: List[str], username: str, api_key: str):
    # Perform the initial comparisons using local methods
    local_comparisons = compare_documents(suspected_text, source_texts)

    # Then, for each source text, perform a Copyscape text search.
    # Here we mock the response as we can't make actual API calls.
    copyscape_results = []
    for source_text in source_texts:
        # This would be an actual API call:
        result = copyscape_text_search(source_text, username=username, api_key=api_key)


        copyscape_results = process_copyscape_response(result)
        copyscape_results.append(result)

    # Combine local comparison results with Copyscape results
    combined_results = []
    for i, (source_text, leven_distance, seq_match, jaccard_sim, cosine_sim) in enumerate(local_comparisons):
        copyscape_plagiarism_detected = copyscape_results[i]["plagiarism_detected"]
        combined_results.append({
            "source_text": source_text,
            "levenshtein_distance": leven_distance,
            "sequence_match_score": seq_match,
            "jaccard_similarity": jaccard_sim,
            "cosine_similarity": cosine_sim,
            "copyscape_plagiarism_detected": copyscape_plagiarism_detected
        })

    return combined_results


def process_copyscape_response(response_json):
    results = []
    for result in response_json.get('result', []):
        copyscape_data = {
            'index': result.get('index'),
            'url': result.get('url'),
            'title': result.get('title'),
            'textsnippet': result.get('textsnippet'),
            'minwordsmatched': result.get('minwordsmatched'),
        }
        if 'wordsmatched' in result:
            copyscape_data.update({
                'wordsmatched': result.get('wordsmatched'),
                'percentmatched': result.get('percentmatched'),
                'textmatched': result.get('textmatched'),
            })
        results.append(copyscape_data)
    return results




# Example usage with mocked responses
USERNAME = ""
API_KEY = ""

plagiarism_results = plagiarism_check_with_copyscape(suspected_text, source_texts, USERNAME, API_KEY)
plagiarism_results


NameError: ignored